# 导包

In [1]:
import torch
import torchvision
import torch.nn
import torch.nn.init
import torch.utils.data as data
# print(torch.__version__)
# print(torchvision.__version__)
# print(torch.cuda.is_available())
import numpy as np
import random

import collections
import re
import nltk

In [2]:
import nltk
import spacy

In [3]:
import data

# 文本预处理简介

一篇文章可以看作字符或单词的序列，常见文本数据的预处理一般包括4个步骤：
1. 读入文本
2. 分词
3. 建立字典，将单词映射成索引
4. 将文本从词的序列转为索引的序列

## STEP1：读入文本

In [4]:
def read_time_machine():
    """
    获取小说《timemachine》
    :return: timechine各行文本组成的列表。
    """
    with open('Datasets/timemachine.txt', 'r') as f:
        lines = [re.sub('[^a-z]+', ' ', line.strip().lower()) for line in f]
    return lines

lines = read_time_machine()
print('# sentences %d' % len(lines))

# sentences 3221


In [5]:
lines[0:2]

['the time machine by h g wells ', '']

## STEP2：分词

In [6]:
def tokenize(sentences, token='word'):
    """
    对文章进行分词，其中文章由句子组成，其实句子分成单词或者字符。
    :param sentences: 文章句子组成的列表。
    :param token: 分词模式，默认按单词分，比标点符号也算单词。
    :return: 列表的列表，分词列表组成句子，句子组成文章。
    """
    if token == 'word':
        return [sentence.split(' ') for sentence in sentences]
    elif token == 'char':
        return [list(sentence) for sentence in sentences]
    else:
        print('ERROR: unkown token type '+token)

sentences = tokenize(lines)
sentences[0:2]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells', ''], ['']]

## STEP3：建立字典

In [7]:
def count_corpus(sentences):
    """
    建立分词集。
    :param sentences: 分好词的句子组成的列表。
    :return: 分词集合。
    """
    tokens = [tk for st in sentences for tk in st]
    return collections.Counter(tokens)

In [8]:
tokens_Collect = count_corpus(sentences)
len(tokens_Collect)

4580

In [9]:
class Vocab(object):
    """
    建立语料库的符号字典。
    * self.idx_to_token：列表，编号 -> 符号。
    * self.token_to_idx：字典，符号 -> 编号。
    * len(vocab)
    * vocab[token]、vocab[token_list]
    * vocab.to_tokens(idx)、vocab.to_tokens(idx_list)
    """
    def __init__(self, sentences, min_freq=0, use_special_tokens=False):
        """
        语料库字典初始化。
        :param tokens: 句子集合。
        :param min_freq: 分词超过该数量才建立索引。
        :param use_special_tokens: 使用特殊含义符号，如空格、句子开始、句子结束，否则只有未知符号。
        """
        counter = count_corpus(sentences)  
        self.token_freqs = list(counter.items())
        self.idx_to_token = []
        if use_special_tokens:
            # padding, begin of sentence, end of sentence, unknown
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            self.idx_to_token += ['', '', '', '']
        else:
            self.unk = 0
            self.idx_to_token += ['']
        self.idx_to_token += [token for token, freq in self.token_freqs if freq >= min_freq and token not in self.idx_to_token]
        self.token_to_idx = dict()
        for idx, token in enumerate(self.idx_to_token):
            self.token_to_idx[token] = idx

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

In [10]:
vocab = Vocab(sentences)
print(list(vocab.token_to_idx.items())[0:10])

[('', 0), ('the', 1), ('time', 2), ('machine', 3), ('by', 4), ('h', 5), ('g', 6), ('wells', 7), ('i', 8), ('traveller', 9)]


## STEP4：将词转换为索引

In [11]:
for i in range(8,10):
    print("words:",sentences[i])
    print("indices:",vocab[sentences[i]])

words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him', '']
indices: [1, 2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0]
words: ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
indices: [20, 21, 22, 23, 24, 16, 25, 26, 27, 28, 29, 30]


## 自然语言工具包
- `spacy`
- `nltk`

In [12]:
text = "Mr. Chen doesn't agree with my suggestion."

### spacy

In [13]:
# python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
print([token.text for token in doc])

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']


### nltk

In [14]:
# nltk.download('punkt')
nltk.tokenize.word_tokenize(text)

['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']

### jieba：中文分词

# 语言模型

一段自然语言文本可以看作是一个离散时间序列，给定一个长度为T的词的序列，语言模型的目标是评估该序列是否合理，即计算该序列的概率。

- 语料库：corpus

语言模型的参数是**词的概率**以及**给定前n个词的情况下的条件概率**。

## n元语法（n-gram）

n元语法通过马尔可夫假设简化模型，马尔可夫假设是指一个词的出现只与前面(n-1个相关，即n阶马尔科夫链。

语言模型的n阶马尔可夫链，即n-gram。

$$
P(w_1, w_2, \ldots, w_T) \approx \prod_{t=1}^T P(w_t \mid w_{t-(n-1)}, \ldots, w_{t-1}) .
$$

### 常用的n-gram模型
- unigram
$$ P(w_1, w_2, w_3, w_4) =  P(w_1) P(w_2) P(w_3) P(w_4) $$
- bigram
$$ P(w_1, w_2, w_3, w_4) =  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_2) P(w_4 \mid w_3) $$
- trigram
$$ P(w_1, w_2, w_3, w_4)  =  P(w_1) P(w_2 \mid w_1) P(w_3 \mid w_1, w_2) P(w_4 \mid w_2, w_3) $$

### n-gram的缺陷
1. 参数空间过大
2. 数据过于稀疏(齐夫定律）

# 时序数据采样
经过预处理后的字符索引序列是连续的，要经过采样才能输入RNN网络。

序列长度为T，时间步数为n，共有$T-n$个合法样本。

- 随机采样
- 相邻采样

**采样后的形状是`（batch_size，num_steps）`**

## 随机采样
随机采样批量大小`batch_size`是每个小批量的样本数，`num_steps`是每个样本所包含的时间步数。

**X似乎必须在间隔为num_steps的分割点上。**

在随机采样中，每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻，因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。

在训练模型时，每次随机采样前都需要重新初始化隐藏状态。

In [15]:
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    """
    时序数据随机采样，样本不重叠。

    :param corpus_indices: 时序数据列表。
    :param batch_size: 批量大小，
    :param num_steps: 采样的时间步数。
    :param device: CUDA / CPU。
    :return: 迭代器（tensorX, tensorY）。
    """
    num_examples = (len(corpus_indices) - 1) // num_steps  
    example_indices = [i * num_steps for i in range(num_examples)] 
    random.shuffle(example_indices)

    def _data(i):
        return corpus_indices[i: i + num_steps]

    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i in range(0, num_examples // batch_size * batch_size, batch_size):
        batch_indices = example_indices[i: i + batch_size] 
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)

In [16]:
my_seq = list(range(31))
# X起始点 0, 6, 12, 18, 24
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[24, 25, 26, 27, 28, 29],
        [18, 19, 20, 21, 22, 23]], device='cuda:0') 
Y: tensor([[25, 26, 27, 28, 29, 30],
        [19, 20, 21, 22, 23, 24]], device='cuda:0') 

X:  tensor([[12, 13, 14, 15, 16, 17],
        [ 6,  7,  8,  9, 10, 11]], device='cuda:0') 
Y: tensor([[13, 14, 15, 16, 17, 18],
        [ 7,  8,  9, 10, 11, 12]], device='cuda:0') 



## 相邻采样

先把时序数据按`batch_size`折叠。

在相邻采样中，一批内的样本间隔相等，相邻的两个随机小批量在原始序列上的位置相毗邻，因此，可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去，在训练模型时，我们只需在每一个迭代周期开始时初始化隐藏状态。

当多个相邻小批量通过传递隐藏状态串联起来时，模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。 为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。

In [17]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    """
    时序数据随机采样，样本不重叠，一批内数据间隔相等，相邻的两个随机小批量在原始序列上的位置相毗邻。

    :param corpus_indices: 时序数据列表。
    :param batch_size: 批量大小，
    :param num_steps: 采样的时间步数。
    :param device: CUDA / CPU。
    :return: 迭代器（tensorX, tensorY）。
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size
    corpus_indices = corpus_indices[: corpus_len]
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [18]:
# 0  6
# 15 21
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]], device='cuda:0') 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]], device='cuda:0') 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]], device='cuda:0') 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]], device='cuda:0') 



# 示例